In [3]:
%%capture
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [4]:
comp = 'titanic'
path = setup_comp(comp, install = 'fastai "timm >= 0.6.2.dev0"')

In [5]:
%%capture
try: import fastai
except ModuleNotFoundError:
    !pip install -Uq fastai

from fastai.imports import *

In [6]:
!ls {path}

gender_submission.csv  test.csv  train.csv


In [7]:
df = pd.read_csv(path/'train.csv', index_col = 'PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
y_train = df['Survived']
X_train = df.drop(['Survived', 'Name'], axis = 1)

In [9]:
from fastai.tabular.all import *

In [10]:
def to_cat(df = df):
    '''
    Convert string-type columns from a dataframe into categorical columns
    '''
    # Identify string/categorical columns in the dataframe
    _, cat = cont_cat_split(df, 1)
    
    # Convert to categorical type using for loops
    for col in cat:
        df[col] = pd.Categorical(df[col])

In [11]:
to_cat(X_train)

In [12]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,female,35.0,1,0,113803,53.1000,C123,S
5,3,male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
%%capture
try: import autosklearn
except ModuleNotFoundError:
    !pip install -Uq auto-sklearn

In [14]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 240,
    per_run_time_limit = 60,
    memory_limit = None,
    n_jobs = -1)

In [15]:
cls.fit(X_train, y_train)

/root/mambaforge/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.metafeatures = self.metafeatures.append(metafeatures)
/root/mambaforge/lib/python3.9/site-packages/autosklearn/metalearning/metalearning/meta_base.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.algorithm_runs[metric].append(runs)


[ERROR] [2022-08-23 15:10:36,573:asyncio.events] 
Traceback (most recent call last):
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1427, in _handle_report
    msgs = await self.scheduler_comm.comm.read()
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/comm/inproc.py", line 211, in read
    raise CommClosedError()
distributed.comm.core.CommClosedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1435, in _handle_report
    await self._reconnect()
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/utils.py", line 799, in wrapper
    return await func(*args, **kwargs)
  File "/root/mambaforge/lib/python3.9/site-packages/distributed/client.py", line 1

AutoSklearnClassifier(memory_limit=None, n_jobs=-1, per_run_time_limit=60,
                      time_left_for_this_task=240)

In [16]:
print(cls.sprint_statistics())

auto-sklearn results:
  Dataset name: 30798895-22f5-11ed-8943-b6f6766cb37b
  Metric: accuracy
  Best validation score: 0.810169
  Number of target algorithm runs: 39
  Number of successful target algorithm runs: 35
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0



In [17]:
print(cls.leaderboard())

          rank  ensemble_weight               type      cost  duration
model_id                                                              
7            1             0.02      liblinear_svc  0.189831  2.510254
10           2             0.02        extra_trees  0.189831  7.985898
4            3             0.08                lda  0.200000  3.639275
17           4             0.02                mlp  0.200000  8.072500
24           5             0.14           adaboost  0.200000  5.511101
35           6             0.04  gradient_boosting  0.200000  2.489987
19           7             0.18           adaboost  0.203390  7.481466
23           8             0.08           adaboost  0.213559  5.564543
27           9             0.04                lda  0.220339  2.586372
29          10             0.04                mlp  0.257627  2.713265
25          11             0.06        extra_trees  0.308475  4.559238
26          12             0.02       bernoulli_nb  0.315254  2.582031
38    

In [18]:
df_test = pd.read_csv(path/'test.csv', index_col = 'PassengerId')
df_test = df_test.drop('Name', axis = 1)
to_cat(df_test)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,330911,7.8292,NaN,Q
893,3,female,47.0,1,0,363272,7.0000,NaN,S
894,2,male,62.0,0,0,240276,9.6875,NaN,Q
895,3,male,27.0,0,0,315154,8.6625,NaN,S
896,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
prediction = cls.predict(df_test)

In [20]:
subm = pd.DataFrame(prediction, index = df_test.index, columns = ['Survived'])

In [21]:
subm.to_csv('subm.csv')
!head subm.csv

PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,0
899,0
900,1


In [22]:
# Submit to competition
from kaggle import api
api.competition_submit_cli('subm.csv', 'autoML 240s - no transformation', comp)

100%|██████████| 2.77k/2.77k [00:00<00:00, 4.99kB/s]


Successfully submitted to Titanic - Machine Learning from Disaster